In [1]:
from tensorflow.keras.models import load_model
import pickle

In [ ]:
## Load the trained model, scaler pickle,onehot
model=load_model('model.h5')


with open("pkl_files/onehot_encoder_geography.pkl", "rb") as file:
    onehot_encoder_geography = pickle.load(file)
    
with open("pkl_files/scaler.pkl", "rb") as file:
    scaler = pickle.load(file)        

In [3]:
#  Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
import pandas as pd
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
 # Label encoder to gender
# input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])
# input_df.head()

input_df["Gender"] = (
    input_df["Gender"]
    .str.strip()
    .str.capitalize()
    .map({"Female": 0, "Male": 1})
)



In [7]:
input_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [11]:
len(input_df["Geography"])


1

In [13]:
# One hot encode
geo_encoded = onehot_encoder_geography.transform([input_df["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geography.get_feature_names_out(["Geography"]))
geo_encoded_df

c:\Users\ps302\OneDrive\Desktop\DL\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
# Concating with the org df
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
# scaling the i/p data
scaled_input_df = scaler.transform(input_df)
scaled_input_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
# Prediction
prediction = model.predict(scaled_input_df)
prediction

1/1 [==============================] - 0s 339ms/step


array([[0.06229775]], dtype=float32)

In [17]:
prediction_proba = prediction[0][0]
if prediction_proba > 0.5 :
    print("The Customer is likely to churn")
else :
    print("The customer is not likely to churn")

The customer is not likely to churn
